<a href="https://colab.research.google.com/github/May-Myat-Noe-Oo/pytorch-youtube-tutorial/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

**Convert the original MNIST image into tensor dimension 4**

In [2]:
transform = transforms.ToTensor()

**Training data from MNIST Datasets**

In [3]:
training_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

100%|██████████| 9.91M/9.91M [00:02<00:00, 4.91MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 131kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.25MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.15MB/s]


**Test data from MNIST Datasets**

In [4]:
test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)

In [5]:
training_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [7]:
pwd

'/content'

In [8]:
ls

sample_data/


In [9]:
cd ../

/


In [10]:
pwd

'/'

In [11]:
ls

bin@                        home/    NGC-DL-CONTAINER-LICENSE  srv/
boot/                       kaggle/  opt/                      sys/
cnn_data/                   lib@     proc/                     tmp/
content/                    lib32@   python-apt/               tools/
cuda-keyring_1.1-1_all.deb  lib64@   python-apt.tar.xz*        usr/
datalab/                    libx32@  root/                     var/
dev/                        media/   run/
etc/                        mnt/     sbin@


**Create batch size for images**

In [12]:
train_loader = DataLoader(training_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

**Define the CNN Model , Two Convolutional Layer**

In [13]:
conv1 = nn.Conv2d(1,6,3,1)
conv2 = nn.Conv2d(6,16,3,1)

**Sample with one image to visualize the flow of convolutional pass**

In [15]:
# Grab 1 MNIST image
for i, (X_Train, y_train) in enumerate(training_data):
  break

In [16]:
X_Train.shape

torch.Size([1, 28, 28])

**One batch size and one image to trace the flow**

In [17]:
x = X_Train.view(1,1,28,28)

**Activation function after layer and inputs are defined**

In [18]:
x = F.relu(conv1(x))

In [19]:
x

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.1549,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.1191,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

**The dimension result of the defined convolution layer one, conv1 = nn.Conv2d(1,6,3,1)**

In [20]:
x.shape

torch.Size([1, 6, 26, 26])

**Pooling the convolution layer one pass**

In [27]:
# kernel of 2 and stride of 2
x = F.max_pool2d(x,2,2)

**Creating the fully convolutional model**

In [30]:
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    # Convolutional layers
    self.conv1 = nn.Conv2d(1,6,3,1)
    self.conv2 = nn.Conv2d(6,16,3,1)

    # Fully Connected Layer
    self.fc1 = nn.Linear(5*5*16,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self, x):
    #Convolutional layer 1
    x=F.relu(self.conv1(x))
    x=F.max_pool2d(x,2,2)  #2*2 kernel and stride 2

    #Convolutional layer 2
    x=F.relu(self.conv2(x))
    x=F.max_pool2d(x,2,2)

    #Re-View to flatten it out
    x=x.view(-1,16*5*5)

    #Fully Connected Layers
    x =F.relu(self.fc1(x))
    x =F.relu(self.fc2(x))
    x =self.fc3(x)

    return F.log_softmax(x,dim=1)



**Create an Instance of our Model**

In [31]:
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

**Define loss function and optimizer for the model**

In [32]:
criterion =nn.CrossEntropyLoss()
optimizer =torch.optim.Adam(model.parameters(),lr=0.001)